In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [ ]:
model_name = "native_transfers"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name],
    root_path_prefix="blockbatch")
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

In [ ]:
print(ctx.client.sql("SHOW TABLES").df())

In [ ]:
# This matches the test's overall count query
block_filter_sql = "(block_number IN (128145990, 128145989) OR block_number % 100 < 2)"

num_native_transfers = ctx.client.sql(
    f"SELECT COUNT(*) as num_native_transfers FROM native_transfers WHERE {block_filter_sql}"
).pl().to_dicts()[0]["num_native_transfers"]
print("num_native_transfers:", num_native_transfers)

In [ ]:
traces_view = input_datasets["ingestion/traces_v1"].create_view()

native_transfers = auxiliary_templates["native_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": traces_view,
    },
)

In [ ]:
native_transfers = (
    native_transfers
    .filter("transfer_type = 'native'")
    .project("*")
)
df = ctx.client.sql(f"SELECT * FROM native_transfers").df()
df.head()